# Semantic Search using the Inference API with the AlibabaCloudSearch service
Learn how to use the [Inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/inference-apis.html) for semantic search.

# Requirements

For this example, you will need:

- An Elastic deployment:
  - we'll be using [Alibaba Elasticsearch](https://www.aliyun.com/product/bigdata/elasticsearch) for this example.
- Elasticsearch 8.16 or above
- A valid API key for the [AlibabaCloud AI Search](https://help.aliyun.com/zh/open-search/search-platform/user-guide/activate-services-and-create-api-key?spm=a2c4g.11186623.0.0.141a2105fiqCEw) is required to use the Inference API with
the AlibabaCloud AI Search service.

# Install packages and connect with Elasticsearch Client

To get started, we'll need to connect to our Elastic deployment using the Python client (version 8.15.0 or above).

First we need to `pip` install the following packages:

- `elasticsearch`

In [ ]:
!pip install elasticsearch

Next, we need to import the modules we need. 

🔐 NOTE: getpass enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [6]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
from getpass import getpass
import json
import time

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their user name, password and elastic host.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [7]:
ELASTIC_USER = getpass("ELASTIC USER: ")

ELASTIC_PASSWORD = getpass("ELASTIC PASSWORD: ")

# e.g.
# hosts = "http://es-xxx.elasticsearch.aliyuncs.com:9200"
hosts = getpass("Host: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    hosts,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
)

Confirm that the client has connected with this test:

In [ ]:
print(client.info())

Refer to [the documentation](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect to a self-managed deployment.

Read [this page](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect using API keys.

<a name="create-the-inference-endpoint"></a>
## Create the inference endpoint

Let's create the inference endpoint by using the [Create inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html).

You'll need a valid API key for AlibabaCloud AI Search for this. You can create one by referring to the [API keys management](https://help.aliyun.com/zh/open-search/search-platform/user-guide/api-keys-management).

In [ ]:
API_KEY = getpass("Enter AlibabaCloud AI Search API key:  ")

# Please enter your AlibabaCloud AI Search host here
# e.g.
# HOST = "default-xxx.opensearch.aliyuncs.com"
HOST = ""

# Please enter your AlibabaCloud AI Search workspace name here
# e.g.
# WORKSPACE = "default"
WORKSPACE = ""

client.inference.put(
    task_type="text_embedding",
    inference_id="os-embeddings-notebook-test",
    body={
        "service": "alibabacloud-ai-search",
        "service_settings": {
            "api_key": API_KEY,
            "service_id": "ops-text-embedding-001",
            "host": HOST,
            "workspace": WORKSPACE,
        },
    },
)

client.inference.put(
    task_type="sparse_embedding",
    inference_id="os-sparse-embeddings-notebook-test",
    body={
        "service": "alibabacloud-ai-search",
        "service_settings": {
            "api_key": API_KEY,
            "service_id": "ops-text-sparse-embedding-001",
            "host": HOST,
            "workspace": WORKSPACE,
        },
    },
)

## Create an ingest pipeline with an inference processor

Create an ingest pipeline with an inference processor by using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method. Reference the inference endpoint created above as the `model_id` to infer against the data that is being ingested in the pipeline.

In [10]:
client.ingest.put_pipeline(
    id="alibaba-embeddings-notebook-test",
    description="Ingest pipeline for alibaba inference notebook test.",
    processors=[
        {
            "inference": {
                "model_id": "os-embeddings-notebook-test",
                "input_output": {
                    "input_field": "plot",
                    "output_field": "plot_embedding",
                },
            }
        },
        {
            "inference": {
                "model_id": "os-sparse-embeddings-notebook-test",
                "input_output": {
                    "input_field": "plot",
                    "output_field": "plot_sparse_embedding",
                },
            }
        },
    ],
)

ObjectApiResponse({'acknowledged': True})

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the inference endpoint to be used. In this example, the model ID is set to `cohere_embeddings`.
- `input_output`: Specifies input and output fields.
- `input_field`: Field name from which the `dense_vector` and `sparse_vector`representation is created.
- `output_field`:  Field name which contains inference results.

## Create index

The mapping of the destination index – the index that contains the embeddings and sparse_embeddings that the model will create based on your input text – must be created. The destination index must have a field with the [dense_vector](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html) field type and a field with the [sparse_vector](https://www.elastic.co/guide/en/elasticsearch/reference/current/sparse-vector.html) field type to index the output of the AlibabaCloud AI Search model.

Let's create an index named `alibaba-text-embeddings-notebook-test` with the mappings we need.

In [11]:
client.indices.delete(
    index="alibaba-text-embeddings-notebook-test", ignore_unavailable=True
)
client.indices.create(
    index="alibaba-text-embeddings-notebook-test",
    settings={"index": {"default_pipeline": "alibaba-embeddings-notebook-test"}},
    mappings={
        "properties": {
            "plot": {"type": "text"},
            "plot_embedding": {"type": "dense_vector", "dims": 1536, "index": "true"},
            "plot_sparse_embedding": {"type": "sparse_vector"},
        }
    },
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'alibaba-text-embeddings-notebook-test'})

## Insert Documents


In [12]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append(
        {
            "_index": "alibaba-text-embeddings-notebook-test",
            "_source": doc,
        }
    )

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `alibaba-text-embeddings-notebook-test` index!")
time.sleep(3)

Done indexing documents into `alibaba-text-embeddings-notebook-test` index!


## Semantic search

### knn search

In [14]:
response = client.search(
    index="alibaba-text-embeddings-notebook-test",
    size=3,
    knn={
        "field": "plot_embedding",
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "os-embeddings-notebook-test",
                "model_text": "Fighting movie",
            }
        },
        "k": 10,
        "num_candidates": 100,
    },
)

for hit in response["hits"]["hits"]:
    doc_id = hit["_id"]
    score = hit["_score"]
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Score: {score}\nTitle: {title}\nPlot: {plot}\n")

Score: 0.6912112
Title: Pulp Fiction
Plot: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Score: 0.66019344
Title: The Usual Suspects
Plot: A sole survivor tells of the twisty events leading up to a horrific gun battle on a boat, which began when five criminals met at a seemingly random police lineup.

Score: 0.64070797
Title: The Dark Knight
Plot: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.



### sparse vector search

In [15]:
response = client.search(
    index="alibaba-text-embeddings-notebook-test",
    query={
        "sparse_vector": {
            "field": "plot_sparse_embedding",
            "inference_id": "os-sparse-embeddings-notebook-test",
            "query": "crime dynasty",
        }
    },
)

for hit in response["hits"]["hits"]:
    doc_id = hit["_id"]
    score = hit["_score"]
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Score: {score}\nTitle: {title}\nPlot: {plot}\n")

Score: 0.1208359
Title: The Godfather
Plot: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.

Score: 0.050445557
Title: Goodfellas
Plot: The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.



**NOTE:** The value of `model_id` in the `query_vector_builder` and the value of `inference_id` in the `sparse_vector` must match the value of `inference_id` you created in the [first step](#create-the-inference-endpoint).